# Exploring and visualizing DICOM GSPS
The purpose of this notebook is to provide guidance when attempting to explore and visualize certain DICOM data sets as provided by [AIDA](https://datahub.aida.scilifelab.se/). In particular, datasets containing DICOM GSPS. For example, the following datasets:
* [CTPA](https://datahub.aida.scilifelab.se/)
* [CTPEL](https://datahub.aida.scilifelab.se/10.23698/aida/ctpel)
* [DRLI](https://datahub.aida.scilifelab.se/10.23698/aida/drli)
* [DRSKE](https://datahub.aida.scilifelab.se/10.23698/aida/drske)

In the following cells, it is assumed that one examination from any of these examinations are available locally.

## Preparations

In [1]:
# Imports
%matplotlib inline
from pathlib import Path
import pydicom
import numpy as np
import matplotlib.pyplot as plt 
from ipywidgets.widgets import * 

from lib.helper import read_and_sort_dicom_files

Specify data to explore and visualize by providing a path to a folder. It is assumed that the specified folder contains a single examination and that all relevant files end with `.dcm`.

In [2]:
folder = Path("/mnt/c/Users/da-for/Downloads/0FC3188AAA7E6851")

Loop over all files to locate GSPS files and to create a mapping structure.

In [3]:
gsps_files = []
sop_uid_to_file = {}
sop_uid_to_series_uid = {}
series_uid_to_sop_uids = {}

for dcm_file in folder.rglob("*.dcm"):
    ds = pydicom.read_file(dcm_file)
    if ds.Modality == "PR":
        gsps_files.append(dcm_file)
    
    sop_uid_to_file[ds.SOPInstanceUID] = dcm_file
    sop_uid_to_series_uid[ds.SOPInstanceUID] = ds.SeriesInstanceUID
    if ds.SeriesInstanceUID not in series_uid_to_sop_uids:
        series_uid_to_sop_uids[ds.SeriesInstanceUID] = []
    series_uid_to_sop_uids[ds.SeriesInstanceUID].append(ds.SOPInstanceUID)

## DICOM GSPS
A DICOM GSPS object contains information about how to present (display) a DICOM instance, a.k.a. an image. This includes aspects such how to rotate, pan and zoom the image, window/level, etc. A GSPS object can also include graphic annotations, i.e., overlays. These can be text objects or graphic objects, and are essentially instructions for a viewer to render vector graphics on top of the referenced image.

In the following, we assume that at least one GSPS file was found and that we use the first one.

In [4]:
gsps_file = gsps_files[0]
gsps_ds = pydicom.read_file(gsps_file)
print(f"The GSPS file {gsps_file} is selected for exploration")

The GSPS file /mnt/c/Users/da-for/Downloads/0FC3188AAA7E6851/im_2/x0000.dcm is selected for exploration


In the following cells, the content of the GSPS file will be explored step by step.

In [5]:
print(f"This GSPS file contains {len(gsps_ds.GraphicAnnotationSequence)} graphic annotations")

This GSPS file contains 901 graphic annotations


The first annotation has the following content:

In [6]:
print(f"The reference to a SOP Instance UID (the image):")
print(gsps_ds.GraphicAnnotationSequence[0].ReferencedImageSequence[0])
print(f"")
print(f"The reference to a Graphic Layer:")
print(gsps_ds.GraphicAnnotationSequence[0].GraphicLayer)
print(f"")
print("The Text Object (if available):")
if "TextObjectSequence" in gsps_ds.GraphicAnnotationSequence[0]:
    print(gsps_ds.GraphicAnnotationSequence[0].TextObjectSequence[0])
print(f"")
print("The Graphic Object (if available):")
if "GraphicObjectSequence" in gsps_ds.GraphicAnnotationSequence[0]:
    print(gsps_ds.GraphicAnnotationSequence[0].GraphicObjectSequence[0])

The reference to a SOP Instance UID (the image):
(0008, 1150) Referenced SOP Class UID            UI: CT Image Storage
(0008, 1155) Referenced SOP Instance UID         UI: 1.3.6.1.4.1.9328.50.4.13397

The reference to a Graphic Layer:
RIGHT FEMUR

The Text Object (if available):

The Graphic Object (if available):
(0070, 0005) Graphic Annotation Units            CS: 'PIXEL'
(0070, 0020) Graphic Dimensions                  US: 2
(0070, 0021) Number of Graphic Points            US: 181
(0070, 0022) Graphic Data                        FL: Array of 362 elements
(0070, 0023) Graphic Type                        CS: 'POLYLINE'


Each annotation can also contain information regarding how to style the annotation, i.e., color, line thickness, filled, font, etc.

More information about the referenced `GraphicLayer` can be found in the `GraphicLayerSequence`.

In [7]:
for graphical_layer in gsps_ds.GraphicLayerSequence:
    if graphical_layer.GraphicLayer == gsps_ds.GraphicAnnotationSequence[0].GraphicLayer:
        print(graphical_layer)

(0070, 0002) Graphic Layer                       CS: 'RIGHT FEMUR'
(0070, 0062) Graphic Layer Order                 IS: '1'
(0070, 0401) Graphic Layer Recommended Display C US: [49512, 38656, 52736]


All this can now be used to visualize the contained annotations, but first we will map all annotations to the respective `ReferencedSOPInstanceUID`.

In [8]:
sop_uid_to_annotations = dict()
SERIES_INSTANCE_UID = None
for annotation in gsps_ds.GraphicAnnotationSequence:
    if not SERIES_INSTANCE_UID:
        SERIES_INSTANCE_UID = sop_uid_to_series_uid[annotation.ReferencedImageSequence[0].ReferencedSOPInstanceUID]
    if annotation.ReferencedImageSequence[0].ReferencedSOPInstanceUID not in sop_uid_to_annotations:
        sop_uid_to_annotations[annotation.ReferencedImageSequence[0].ReferencedSOPInstanceUID] = []
    sop_uid_to_annotations[annotation.ReferencedImageSequence[0].ReferencedSOPInstanceUID].append(annotation)

sop_uids = series_uid_to_sop_uids[SERIES_INSTANCE_UID]
dcm_files = [sop_uid_to_file[sop_uid] for sop_uid in sop_uids]
sorted_ds = read_and_sort_dicom_files(dcm_files)

Visualize the annotations together with the referenced images.


In [9]:
# Define a function to use to show images and associated annotations
def show_image_and_annotations(ind, sorted_ds, sop_uid_to_annotations):
    image_ds = sorted_ds[ind]
    sop_uid = image_ds.SOPInstanceUID
    fig, ax = plt.subplots(1, figsize = (9,9), dpi=100)
    ax.imshow(image_ds.pixel_array, cmap=plt.cm.gray)
    annotations = []
    if sop_uid in sop_uid_to_annotations:
        annotations = sop_uid_to_annotations[sop_uid]
    for annotation in annotations:
        if "GraphicObjectSequence" in annotation:
            coords = np.reshape(annotation.GraphicObjectSequence[0].GraphicData, (-1,2))
            annotation_type = annotation.GraphicObjectSequence[0].GraphicType
            filled = annotation.GraphicObjectSequence[0].get("GraphicFilled", "N") == "Y"
            if annotation_type == "POINT":
                # Instead of a point we draw a circle
                poly = plt.Circle(coords[0], radius=3, facecolor=None, fill=filled, color=[1,0,0])
                ax.add_patch(poly)
            elif annotation_type == "POLYLINE":
                poly = plt.Polygon(coords, facecolor=None, fill=filled, color=[1,0,0])
                ax.add_patch(poly)
            elif annotation_type == "INTERPOLATED":
                lines = plt.Line2D(coords[:,0], coords[:,1], color=[1,0,0])
                ax.add_line(lines)
            elif annotation_type == "CIRCLE":
                poly = plt.Circle(
                    xy=coords[0], 
                    radius=np.linalg.norm(coords[1] - coords[0]), 
                    facecolor=None, 
                    fill=filled, 
                    color=[1,0,0])
                ax.add_patch(poly)
            elif annotation_type == "ELLIPSE":
                # TODO This part has not been tested
                major_axis = coords[1] - coords[0]
                major_axis = major_axis / np.linalg.norm(major_axis)
                x_vec = np.array([1, 0])       
                angle_1 = np.arctan2(*x_vec[::-1])
                angle_2 = np.arctan2(*major_axis[::-1])
                rot_angle = np.rad2deg((angle_1 - angle_2) % (2 * np.pi))
                poly = plt.Circle(
                    xy=coords[1] - coords[0],
                    width=np.linalg.norm(coords[1] - coords[0]),
                    height=np.linalg.norm(coords[3] - coords[2]),
                    angle=rot_angle,
                    facecolor=None, 
                    fill=filled, 
                    color=[1,0,0])
                ax.add_patch(poly)
                pass
        elif "TextObjectSequence" in annotation:
            coords = annotation.TextObjectSequence[0].AnchorPoint
            text = annotation.TextObjectSequence[0].UnformattedTextValue
            ax.text(coords[0], coords[1], text, color=[1,0,0])
            
    plt.show()
    return ind

In [10]:
interact(
    show_image_and_annotations, 
    ind=(0, len(sorted_ds)-1), 
    sorted_ds=fixed(sorted_ds), 
    sop_uid_to_annotations=fixed(sop_uid_to_annotations),
)

interactive(children=(IntSlider(value=267, description='ind', max=534), Output()), _dom_classes=('widget-inter…

<function __main__.show_image_and_annotations(ind, sorted_ds, sop_uid_to_annotations)>